In [2]:
import pandas as pd
import optuna
import yfinance as yf
import pandas_datareader.data as web
from datetime import datetime
import numpy as np
import pandas_ta as ta
from keras.optimizers import Adam
from keras.optimizers import SGD
from keras.optimizers import Nadam
from sklearn.metrics import confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.regularizers import l2
from keras.callbacks import EarlyStopping

In [4]:
pip list

Package                       Version
----------------------------- ---------------
absl-py                       2.1.0
aiobotocore                   2.5.0
aiofiles                      22.1.0
aiohttp                       3.8.5
aioitertools                  0.7.1
aiosignal                     1.2.0
aiosqlite                     0.18.0
alabaster                     0.7.12
alembic                       1.13.1
anaconda-anon-usage           0.4.2
anaconda-catalogs             0.2.0
anaconda-client               1.12.1
anaconda-cloud-auth           0.1.3
anaconda-navigator            2.5.0
anaconda-project              0.11.1
anyio                         3.5.0
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arrow                         1.2.3
astroid                       2.14.2
astropy                       5.1
asttokens                     2.0.5
astunparse                    1.6.3
async-timeout                 4.0.2
atomicwri

In [2]:
import warnings
warnings.filterwarnings('ignore', category = pd.errors.PerformanceWarning)

In [3]:
# Pobieramy dane dotyczące BTC z yahoo
BTC = yf.download('BTC-USD', start = '2018-01-01', end = '2019-12-31')

BTC['Target'] = (BTC['Close'].diff() > 0).astype(int)

def add_lagged_features(data, column_name, n_days):
    for i in range(1, n_days + 1):
        lagged_column_name = f'{column_name}_{i}d_back'
        data[lagged_column_name] = data[column_name].shift(i)
    return data

BTC = add_lagged_features(BTC, 'Close', 30)

# Usuwamy pierwszych 30 wierszy
BTC = BTC.dropna().reset_index(drop = True)

# print(BTC.head())
# print(len(BTC))
# print(BTC.shape)

[*********************100%%**********************]  1 of 1 completed


In [4]:
# Konwersacja zmiennej Target na zmienną binarną
target = to_categorical(BTC['Target'].values)

# Usuwamy niepotrzebne kolumny
features = BTC.drop(['Target', 'Adj Close'], axis = 1).values 

# Dzielimy dane na treningowe i testowe
split_idx = int(len(features) * 0.8)

m = int(np.floor(0.8 * len(features)))
train_features = features[:m]
test_features = features[m:]
train_target = target[:m]
test_target = target[m:]

In [5]:
##################################################
##################################################
# WERSJA MAŁO SKOMPLIKOWANA
##################################################
##################################################

In [6]:
# Budujemy model
model = Sequential([
    Dense(64, activation = 'relu', input_shape = (train_features.shape[1],)),
    Dense(64, activation = 'relu'),
    Dense(2, activation = 'softmax')
])

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

# Trenujemy model
history = model.fit(train_features, train_target, epochs = 50, batch_size = 12, validation_split = 0.2)

C:\Users\48798\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.5555 - loss: 289916608.0000 - val_accuracy: 0.5804 - val_loss: 205725520.0000
Epoch 2/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4886 - loss: 30193556.0000 - val_accuracy: 0.5804 - val_loss: 35707524.0000
Epoch 3/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5093 - loss: 18281562.0000 - val_accuracy: 0.5804 - val_loss: 14374555.0000
Epoch 4/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5170 - loss: 6873925.5000 - val_accuracy: 0.4196 - val_loss: 2064126.2500
Epoch 5/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5018 - loss: 3712887.5000 - val_accuracy: 0.4196 - val_loss: 20624412.0000
Epoch 6/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4531 - loss: 18346874.0000 - val_accuracy: 0.4196 - val_loss: 17878398.0000
Epoch 7/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4663 - loss: 11162638.0000 - val_accuracy: 0.4196 - val_loss: 51883192.0000
Epoch 8/50
38/38 ━━━

38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5331 - loss: 6838606.0000 - val_accuracy: 0.4196 - val_loss: 46633244.0000
Epoch 49/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4873 - loss: 7266934.0000 - val_accuracy: 0.4196 - val_loss: 26787246.0000
Epoch 50/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4798 - loss: 6724818.5000 - val_accuracy: 0.5804 - val_loss: 25634406.0000


In [7]:
# Predykcja na zbiorze treningowym
train_predictions = model.predict(train_features)
train_predictions = np.argmax(train_predictions, axis = 1)
train_true = np.argmax(train_target, axis = 1)

# Obliczanie macierzy pomyłek
cm = confusion_matrix(train_true, train_predictions)
TP = cm[1, 1]  # True Positives
TN = cm[0, 0]  # True Negatives
FP = cm[0, 1]  # False Positives
FN = cm[1, 0]  # False Negatives

print(TP)
print(TN)
print(FP)
print(FN)

# TPR, FPR, TNR, PPV, NPV
ACC = (TP + TN) / (TP + TN + FP + FN)
TPR = TP / (TP + FN)
FPR = FP / (FP + TN)
TNR = TN / (TN + FP)
PPV = TP / (TP + FP)
NPV = TN / (TN + FN)

print('Accuracy (ACC):', ACC)
print('True Positive Rate (TPR):', TPR)
print('False Positive Rate (FPR):', FPR)
print('True Negative Rate (TNR):', TNR)
print('Positive Predictive Value (PPV):', PPV)
print('Negative Predictive Value (NPV):', NPV)

18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
303
0
256
0
Accuracy (ACC): 0.5420393559928444
True Positive Rate (TPR): 1.0
False Positive Rate (FPR): 1.0
True Negative Rate (TNR): 0.0
Positive Predictive Value (PPV): 0.5420393559928444
Negative Predictive Value (NPV): nan


C:\Users\48798\AppData\Local\Temp\ipykernel_18728\1020962155.py:24: RuntimeWarning: invalid value encountered in scalar divide
  NPV = TN / (TN + FN)


In [8]:
# Predykcja na zbiorze testowym
test_predictions = model.predict(test_features)
test_predictions = np.argmax(test_predictions, axis = 1)
test_true = np.argmax(test_target, axis = 1)

# Obliczanie macierzy pomyłek
cm = confusion_matrix(test_true, test_predictions)
TP = cm[1, 1]  # True Positives
TN = cm[0, 0]  # True Negatives
FP = cm[0, 1]  # False Positives
FN = cm[1, 0]  # False Negatives

print(TP)
print(TN)
print(FP)
print(FN)

# TPR, FPR, TNR, PPV, NPV
ACC = (TP + TN) / (TP + TN + FP + FN)
TPR = TP / (TP + FN)
FPR = FP / (FP + TN)
TNR = TN / (TN + FP)
PPV = TP / (TP + FP)
NPV = TN / (TN + FN)

print('Accuracy (ACC):', ACC)
print('True Positive Rate (TPR):', TPR)
print('False Positive Rate (FPR):', FPR)
print('True Negative Rate (TNR):', TNR)
print('Positive Predictive Value (PPV):', PPV)
print('Negative Predictive Value (NPV):', NPV)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
59
0
81
0
Accuracy (ACC): 0.42142857142857143
True Positive Rate (TPR): 1.0
False Positive Rate (FPR): 1.0
True Negative Rate (TNR): 0.0
Positive Predictive Value (PPV): 0.42142857142857143
Negative Predictive Value (NPV): nan


C:\Users\48798\AppData\Local\Temp\ipykernel_18728\4228670605.py:24: RuntimeWarning: invalid value encountered in scalar divide
  NPV = TN / (TN + FN)


In [9]:
##################################################
##################################################
##################################################
##################################################

In [10]:
##################################################
##################################################
# WERSJA BARDZIEJ SKOMPLIKOWANA
##################################################
##################################################

In [11]:
# Korzystamy z biblioteki OPTUNA, która służy do automatycznego dostrajania hiperparametrów
def create_model(trial):
    # Liczba warstw ukrytych, od 1 do 3
    n_layers = trial.suggest_int('n_layers', 1, 3)
    model = Sequential()
    # Dodanie pierwszej warstwy ukrytej
    model.add(Dense(trial.suggest_int('n_units_first', 10, 300), activation = 'relu', input_shape = (train_features.shape[1],)))
    # Dodanie warstwy Dropout
    model.add(Dropout(trial.suggest_float('dropout_first', 0.0, 0.5)))

    # Dodawanie kolejnych warstw ukrytych w pętli zależnie od liczby warstw
    for i in range(n_layers):
        model.add(Dense(trial.suggest_int(f'n_units_{i}', 10, 300), activation = 'relu'))
        model.add(Dropout(trial.suggest_float(f'dropout_{i}', 0.0, 0.5)))

    # Dodanie warstwy wyjściowej
    model.add(Dense(2, activation = 'softmax'))
    # Sugestia Optuna dotycząca współczynnika uczenia dla optymalizatora Adama
    lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
    model.compile(optimizer = Adam(learning_rate = lr), loss = 'categorical_crossentropy', metrics = ['accuracy'])
    return model

# Definicja funkcji celu dla procesu dostrajania Optuna
def objective(trial):
    # Tworzenie modelu z bieżącymi parametrami sugerowanymi przez Optuna
    model = create_model(trial)
    # Ustawienie mechanizmu wczesnego zatrzymywania treningu w celu uniknięcia przeuczenia
    early_stopping = EarlyStopping(monitor = 'val_loss', patience = 10, verbose = 1, restore_best_weights = True)
    model.fit(train_features, train_target, epochs = 50, batch_size = 32, validation_split = 0.2, 
              callbacks = [early_stopping], verbose = 0)
    _, accuracy = model.evaluate(test_features, test_target, verbose = 0)
    return accuracy

# Uruchomienie procesu dostrajania
study = optuna.create_study(direction = 'maximize')
study.optimize(objective, n_trials = 10)

# Wyświetlenie najlepszych parametrów znalezionych przez Optuna
print('Najlepsze parametry:', study.best_trial.params)

[I 2024-06-06 11:34:06,936] A new study created in memory with name: no-name-56a8812d-0153-480f-aae5-4fdf49201c0d
C:\Users\48798\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\48798\AppData\Local\Temp\ipykernel_18728\3357861014.py:19: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)


Epoch 11: early stopping
Restoring model weights from the end of the best epoch: 1.


[I 2024-06-06 11:34:12,997] Trial 0 finished with value: 0.5785714387893677 and parameters: {'n_layers': 3, 'n_units_first': 31, 'dropout_first': 0.4020311774629955, 'n_units_0': 65, 'dropout_0': 0.36696993887986223, 'n_units_1': 57, 'dropout_1': 0.3239385461419459, 'n_units_2': 207, 'dropout_2': 0.13897675723354436, 'lr': 0.00016983022705747668}. Best is trial 0 with value: 0.5785714387893677.


Epoch 12: early stopping
Restoring model weights from the end of the best epoch: 2.


[I 2024-06-06 11:34:17,371] Trial 1 finished with value: 0.4214285612106323 and parameters: {'n_layers': 1, 'n_units_first': 78, 'dropout_first': 0.12855168520696664, 'n_units_0': 96, 'dropout_0': 0.25466652483637653, 'lr': 0.09797977484479845}. Best is trial 0 with value: 0.5785714387893677.


...

Epoch 20: early stopping
Restoring model weights from the end of the best epoch: 10.


[I 2024-06-06 11:35:06,132] Trial 8 finished with value: 0.4214285612106323 and parameters: {'n_layers': 2, 'n_units_first': 46, 'dropout_first': 0.31426480690289627, 'n_units_0': 169, 'dropout_0': 0.18256603106083102, 'n_units_1': 268, 'dropout_1': 0.2488469990916516, 'lr': 0.029770351078291787}. Best is trial 0 with value: 0.5785714387893677.


Epoch 18: early stopping
Restoring model weights from the end of the best epoch: 8.


[I 2024-06-06 11:35:12,557] Trial 9 finished with value: 0.5785714387893677 and parameters: {'n_layers': 2, 'n_units_first': 77, 'dropout_first': 0.010457280380750777, 'n_units_0': 114, 'dropout_0': 0.141733376723098, 'n_units_1': 218, 'dropout_1': 0.05000470003294177, 'lr': 0.00023130600465818155}. Best is trial 0 with value: 0.5785714387893677.


Najlepsze parametry: {'n_layers': 3, 'n_units_first': 31, 'dropout_first': 0.4020311774629955, 'n_units_0': 65, 'dropout_0': 0.36696993887986223, 'n_units_1': 57, 'dropout_1': 0.3239385461419459, 'n_units_2': 207, 'dropout_2': 0.13897675723354436, 'lr': 0.00016983022705747668}


In [12]:
best_params = {'n_layers': 2, 
               'n_units_first': 281, 
               'dropout_first': 0.23922171710238138, 
               'n_units_0': 255, 
               'dropout_0': 0.11460593371482963, 
               'n_units_1': 85, 
               'dropout_1': 0.4701359447843255, 
               'lr': 1.7415110409580812e-05}

# Budujemy model sieci neuronowej
model = Sequential()
model.add(Dense(best_params['n_units_first'], activation = 'relu', input_shape = (train_features.shape[1],)))
model.add(Dropout(best_params['dropout_first']))

model.add(Dense(best_params['n_units_0'], activation = 'relu'))
model.add(Dropout(best_params['dropout_0']))

model.add(Dense(best_params['n_units_1'], activation = 'relu'))
model.add(Dropout(best_params['dropout_1']))

# model.add(Dense(best_params['n_units_2'], activation = 'relu'))
# model.add(Dropout(best_params['dropout_2']))

model.add(Dense(2, activation = 'softmax'))

# Optymalizator Adam
optimizer = Adam(learning_rate = best_params['lr'])

model.compile(optimizer = optimizer, loss = 'categorical_crossentropy', metrics = ['accuracy'])

# Dodajemy funkcję, która przerwie trenowanie modelu, gdy model przestaje poprawiać swoją wydajność na zbiorze walidacyjnym
early_stopping = EarlyStopping(monitor = 'val_loss',  
                               patience = 10,         # liczba epok bez poprawy po której trening zostanie zatrzymany
                               verbose = 1,           
                               restore_best_weights = True) 


# Trenowanie modelu z dodaniem callbacku
history = model.fit(train_features, train_target, 
                    epochs = 100, 
                    batch_size = 32, 
                    validation_split = 0.2,
                    callbacks = [early_stopping]) 

# Ocena modelu na danych testowych
test_performance = model.evaluate(test_features, test_target)
print('Test loss:', test_performance[0])
print('Test accuracy:', test_performance[1])

# Ocena modelu na danych treningowych
train_performance = model.evaluate(train_features, train_target)
print('Train loss:', train_performance[0])
print('Train accuracy:', train_performance[1])

Epoch 1/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 3s 36ms/step - accuracy: 0.5558 - loss: 293394944.0000 - val_accuracy: 0.5804 - val_loss: 637074496.0000
Epoch 2/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5138 - loss: 286252992.0000 - val_accuracy: 0.5804 - val_loss: 491871136.0000
Epoch 3/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5153 - loss: 262996096.0000 - val_accuracy: 0.5804 - val_loss: 370723424.0000
Epoch 4/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.4811 - loss: 289806400.0000 - val_accuracy: 0.5804 - val_loss: 283448448.0000
Epoch 5/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4400 - loss: 316305568.0000 - val_accuracy: 0.5804 - val_loss: 229229024.0000
Epoch 6/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.4684 - loss: 265413728.0000 - val_accuracy: 0.5804 - val_loss: 184120752.0000
Epoch 7/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.4717 - loss: 275112320.0000 - val_accuracy: 0.5804 - val_loss: 145519520.00

In [13]:
# Predykcja na zbiorze treningowym
train_predictions = model.predict(train_features)
train_predictions = np.argmax(train_predictions, axis = 1)
train_true = np.argmax(train_target, axis = 1)

# Obliczanie macierzy pomyłek
cm = confusion_matrix(train_true, train_predictions)
TP = cm[1, 1]  # True Positives
TN = cm[0, 0]  # True Negatives
FP = cm[0, 1]  # False Positives
FN = cm[1, 0]  # False Negatives

print(TP)
print(TN)
print(FP)
print(FN)

# TPR, FPR, TNR, PPV, NPV
ACC = (TP + TN) / (TP + TN + FP + FN)
TPR = TP / (TP + FN)
FPR = FP / (FP + TN)
TNR = TN / (TN + FP)
PPV = TP / (TP + FP)
NPV = TN / (TN + FN)

print('Accuracy (ACC):', ACC)
print('True Positive Rate (TPR):', TPR)
print('False Positive Rate (FPR):', FPR)
print('True Negative Rate (TNR):', TNR)
print('Positive Predictive Value (PPV):', PPV)
print('Negative Predictive Value (NPV):', NPV)

18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
0
256
0
303
Accuracy (ACC): 0.45796064400715564
True Positive Rate (TPR): 0.0
False Positive Rate (FPR): 0.0
True Negative Rate (TNR): 1.0
Positive Predictive Value (PPV): nan
Negative Predictive Value (NPV): 0.45796064400715564


C:\Users\48798\AppData\Local\Temp\ipykernel_18728\1020962155.py:23: RuntimeWarning: invalid value encountered in scalar divide
  PPV = TP / (TP + FP)


In [14]:
# Predykcja na zbiorze testowym
test_predictions = model.predict(test_features)
test_predictions = np.argmax(test_predictions, axis = 1)
test_true = np.argmax(test_target, axis = 1)

# Obliczanie macierzy pomyłek
cm = confusion_matrix(test_true, test_predictions)
TP = cm[1, 1]  # True Positives
TN = cm[0, 0]  # True Negatives
FP = cm[0, 1]  # False Positives
FN = cm[1, 0]  # False Negatives

print(TP)
print(TN)
print(FP)
print(FN)

# TPR, FPR, TNR, PPV, NPV
ACC = (TP + TN) / (TP + TN + FP + FN)
TPR = TP / (TP + FN)
FPR = FP / (FP + TN)
TNR = TN / (TN + FP)
PPV = TP / (TP + FP)
NPV = TN / (TN + FN)

print('Accuracy (ACC):', ACC)
print('True Positive Rate (TPR):', TPR)
print('False Positive Rate (FPR):', FPR)
print('True Negative Rate (TNR):', TNR)
print('Positive Predictive Value (PPV):', PPV)
print('Negative Predictive Value (NPV):', NPV)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
0
81
0
59
Accuracy (ACC): 0.5785714285714286
True Positive Rate (TPR): 0.0
False Positive Rate (FPR): 0.0
True Negative Rate (TNR): 1.0
Positive Predictive Value (PPV): nan
Negative Predictive Value (NPV): 0.5785714285714286


C:\Users\48798\AppData\Local\Temp\ipykernel_18728\4228670605.py:23: RuntimeWarning: invalid value encountered in scalar divide
  PPV = TP / (TP + FP)
